In [1]:
import pandas as pd
import numpy as np
import pybea
import re

In [2]:
meta_data = pybea.get_data_set_list()

print meta_data

                      DatasetDescription         DatasetName
0    Retrieves various Regional datasets        RegionalData
1                   Standard NIPA tables                NIPA
2   Standard NI underlying detail tables  NIUnderlyingDetail
3              Multinational Enterprises                 MNE
4           Standard Fixed Assets tables         FixedAssets
5    International Transactions Accounts                 ITA
6      International Investment Position                 IIP
7                        GDP by Industry       GDPbyIndustry
8     Detailed Regional Income data sets      RegionalIncome
9    Detailed Regional Product data sets     RegionalProduct
10                     Input-Output Data         InputOutput


In [3]:
# access the BEA data API...
meta_data = pybea.get_parameter_list(DataSetName='RegionalData')

# ...display the resulting Pandas DataFrame
print(meta_data)

  AllValue  MultipleAcceptedFlag ParameterDataType ParameterDefaultValue  \
0                              0            string                         
1                              1            string                         
2      ALL                     1           integer                   ALL   

                      ParameterDescription  ParameterIsRequiredFlag  \
0  The code of the key statistic requested                        1   
1                             GeoFips Code                        0   
2                                     Year                        0   

  ParameterName  
0       KeyCode  
1       GeoFips  
2          Year  


In [5]:
# access the BEA data API...
regional_keys_df = pybea.get_parameter_values(DataSetName='RegionalData',
                                       ParameterName='KeyCode')


#print regional_keys_df[regional_keys_df['KeyCode'].str.contains('RPI')]
rpi_search = regional_keys_df[regional_keys_df['KeyCode'].str.contains('RPI')]['Description']
for description in rpi_search:
    print description
    
pi_search = regional_keys_df[regional_keys_df['Description'].str.contains('personal income')]
for des,key in zip(pi_search['Description'],pi_search['KeyCode']):
    print des,key



Real personal income (state regional price parities)
Real personal income (MSA regional price parities)
Total personal income (state annual income) TPI_SI
Per capita personal income (state annual income) PCPI_SI
Nonfarm personal income (state annual income) NFPI_SI
Disposable personal income (state annual income) DPI_SI
Per capita disposable personal income (dollars) (state annual income) PCDPI_SI
Total personal income (county annual income) TPI_CI
Per capita personal income (county annual income) PCPI_CI
Nonfarm personal income (county annual income) NFPI_CI
Total personal income (MSA annual income) TPI_MI
Per capita personal income (MSA annual income) PCPI_MI
Nonfarm personal income (MSA annual income) NFPI_MI
Total personal income (state quarterly income) TPI_QI
Nonfarm personal income (state quarterly income) NFPI_QI
Real personal income (state regional price parities) RPI_SI
Per capita real personal income (state regional price parities) RPCPI_SI
Real personal income (MSA regional

In [6]:
data = [0 for i in range(len(regional_keys_df))]

for key,des,i in zip(regional_keys_df['KeyCode'],regional_keys_df['Description'],range(len(regional_keys_df))):

    data[i] = pybea.get_data(DataSetName='RegionalData',
                          KeyCodes=[key],
                          )

    print i,len(data[i]['TimePeriod'].unique()), '%12s'%key, des

0 18       GDP_SP GDP in current dollars (state annual product)
1 18      RGDP_SP Real GDP in chained dollars (state annual product)
2 18    PCRGDP_SP Per capita real GDP (state annual product)
3 18      COMP_SP Compensation of employees (state annual product)
4 18    TOPILS_SP Taxes on production and imports less subsidies (state annual product)
5 18       GOS_SP Gross operating surplus (state annual product)
6 18      SUBS_SP Subsidies (state annual product)
7 18      TOPI_SP Taxes on production and imports (state annual product)
8 13       GDP_MP GDP in current dollars (MSA annual product)
9 13      RGDP_MP Real GDP in chained dollars (MSA annual product)
10 13    PCRGDP_MP Per capita real GDP (MSA annual product)
11 86       TPI_SI Total personal income (state annual income)
12 86       POP_SI Population (state annual income)
13 86      PCPI_SI Per capita personal income (state annual income)
14 86      NFPI_SI Nonfarm personal income (state annual income)
15 86       FPI_SI Farm i

In [55]:
data = pybea.get_data(DataSetName='RegionalData',
                      KeyCodes=['PCPI_MI'],
                      GeoFips='MSA',
                      )

In [57]:
data['TimePeriod'].unique()

array([1969, 1970, 1971, 1972, 1973, 1974, 2011, 2012, 2013, 2005, 2006,
       2007, 2008, 2009, 2010, 1999, 2000, 2001, 2002, 2003, 2004, 1993,
       1994, 1995, 1996, 1997, 1998, 1987, 1988, 1989, 1990, 1991, 1992,
       1981, 1982, 1983, 1984, 1985, 1986, 1975, 1976, 1977, 1978, 1979,
       1980])

In [47]:
# access the BEA data API...
regincome_data = pybea.get_parameter_list(DataSetName='RegionalIncome')

# ...display the resulting Pandas DataFrame
print(regincome_data)

   MultipleAcceptedFlag ParameterDataType ParameterDefaultValue  \
0                     1            string                   NaN   
1                     0           integer                   NaN   
2                     0            string                         
3                     1            string                 LAST5   

                                ParameterDescription  ParameterIsRequiredFlag  \
0  Comma-delimited list of 5-character geographic...                        1   
1              Line code for a statistic or industry                        1   
2             Income or employment table to retrieve                        1   
3  Comma-delimted list of years; LAST5 for latest...                        0   

  ParameterName  
0       GeoFips  
1      LineCode  
2     TableName  
3          Year  


In [59]:
regincome_keys_df = pybea.get_parameter_values(DataSetName='RegionalIncome',
                                       ParameterName='TableName')

only_industry_specific_df = regincome_keys_df[~regincome_keys_df['Desc'].str.contains('Non-Industry')]
personal_income_df = only_industry_specific_df[regincome_keys_df['Desc'].str.contains('Personal Income')]
for des,key in zip(only_industry_specific_df['Desc'],only_industry_specific_df['Key']):
    print des,key

Total Full-Time and Part-Time Employment by SIC Industry (SIC) [A] CA25
Total Full-Time and Part-Time Employment by NAICS Industry (NAICS) [A] CA25N
Personal Income by Major Component and Earnings by SIC Industry (SIC) [A] CA5
Personal Income by Major Component and Earnings by NAICS Industry (NAICS) [A] CA5N
Compensation of Employees by SIC Industry (SIC) [A] CA6
Compensation of Employees by NAICS Industry (NAICS) [A] CA6N
Total Full-Time and Part-Time Employment by SIC Industry (SIC) [A] SA25
Total Full-Time and Part-Time Employment by NAICS Industry (NAICS) [A] SA25N
Full-Time and Part-Time Wage and Salary Employment by SIC Industry (SIC) [A] SA27
Full-Time and Part-Time Wage and Salary Employment by NAICS Industry (NAICS) [A] SA27N
Personal Income by Major Component and Earnings by SIC Industry (SIC) [A] SA5
Personal Income by Major Component and Earnings by Industry (Historical) (SIC) [A] SA5H
Personal Income by Major Component and Earnings by NAICS Industry (NAICS) [A] SA5N
Compen

/usr/local/lib/python2.7/dist-packages/pandas/core/frame.py:1825: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


In [48]:
url = 'http://bea.gov/api/data/?UserID=2A4FC9C7-5523-4C19-B363-3AA5B5426339&method=GetParameterValues'\
    'Filtered&datasetname=RegionalIncome&TargetParameter=LineCode&TableName=CA5&ResultFormat=JSON'
    
raw_json = json.loads(rq.get(url).content.decode('latin1'))
json_request = raw_json['BEAAPI']['Request']
json_result = raw_json['BEAAPI']['Results']


df = pd.DataFrame(json_result['ParamValue'])
print 'RegionalIncome Line Codes'
for i in range(len(df)):
    print df.iloc[i]['Key'], df.iloc[i]['Desc']

RegionalIncome Line Codes
10 [CA5] Personal income
20 [CA5] Population (persons)
30 [CA5] Per capita personal income (dollars)
35 [CA5] Earnings by place of work
36 [CA5] less: Contributions for government social insurance
37 [CA5] Employee and self-employed contributions for government social insurance
38 [CA5] Employer contributions for government social insurance
42 [CA5] plus: Adjustment for residence
45 [CA5] equals: Net earnings by place of residence
46 [CA5] plus: Dividends, interest, and rent
47 [CA5] plus: Personal current transfer receipts
50 [CA5] Wages and salaries
60 [CA5] Supplements to wages and salaries
61 [CA5] Employer contributions for employee pension and insurance funds
62 [CA5] Employer contributions for government social insurance
70 [CA5] Proprietors' income
71 [CA5] Farm proprietors' income
72 [CA5] Nonfarm proprietors' income
81 [CA5] Farm earnings
82 [CA5] Nonfarm earnings
90 [CA5] Private nonfarm earnings
100 [CA5] Agricultural services, forestry, and fishin

In [58]:
url = 'http://bea.gov/api/data/?UserID=2A4FC9C7-5523-4C19-B363-3AA5B5426339&method=GetParameterValues'\
'Filtered&datasetname=RegionalIncome&TargetParameter=Year&TableName=CA5&ResultFormat=json'
    
raw_json = json.loads(rq.get(url).content.decode('latin1'))
json_request = raw_json['BEAAPI']['Request']
json_result = raw_json['BEAAPI']['Results']


df = pd.DataFrame(json_result['ParamValue'])
df

,Desc,Key
0,1969,1969
1,1970,1970
2,1971,1971
3,1972,1972
4,1973,1973
5,1974,1974
6,1975,1975
7,1976,1976
8,1977,1977
9,1978,1978


In [1]:
import requests as rq
import json

def do_request(arguments):
    base_str = 'http://bea.gov/api/data/?UserID=2A4FC9C7-5523-4C19-B363-3AA5B5426339'
    
    url = base_str+arguments
    
    print(url)
    
    raw_json = json.loads(rq.get(url).content.decode('latin1'))
    json_request = raw_json['BEAAPI']['Request']
    json_result = raw_json['BEAAPI']['Results']
    
    return json_request,json_result
    


In [3]:
args = '&method=GetParameterValuesFiltered&datasetname=RegionalProduct&TargetParameter=IndustryId&Component=GDP_SAS&ResultFormat=json'

req,res = do_request(args)

print res

http://bea.gov/api/data/?UserID=2A4FC9C7-5523-4C19-B363-3AA5B5426339&method=GetParameterValuesFiltered&datasetname=RegionalProduct&TargetParameter=IndustryId&Component=GDP_SAS&ResultFormat=json
{u'ParamValue': [{u'Key': u'1', u'Desc': u'All industry total [State Annual SIC]'}, {u'Key': u'2', u'Desc': u'Private industries [State Annual SIC]'}, {u'Key': u'3', u'Desc': u'Agriculture, forestry, and fishing [State Annual SIC]'}, {u'Key': u'4', u'Desc': u'Farms [State Annual SIC]'}, {u'Key': u'5', u'Desc': u'Agricultural services, forestry, and fishing [State Annual SIC]'}, {u'Key': u'6', u'Desc': u'Mining [State Annual SIC]'}, {u'Key': u'7', u'Desc': u'Metal mining [State Annual SIC]'}, {u'Key': u'8', u'Desc': u'Coal mining [State Annual SIC]'}, {u'Key': u'9', u'Desc': u'Oil and gas extraction [State Annual SIC]'}, {u'Key': u'10', u'Desc': u'Nonmetallic minerals, except fuels [State Annual SIC]'}, {u'Key': u'11', u'Desc': u'Construction [State Annual SIC]'}, {u'Key': u'12', u'Desc': u'Manuf